# Fundamentals of Information Systems

## Python Programming (for Data Science)

### Master's Degree in Data Science

#### Giorgio Maria Di Nunzio
#### (Courtesy of Gabriele Tolomei FIS 2018-2019)
<a href="mailto:giorgiomaria.dinunzio@unipd.it">giorgiomaria.dinunzio@unipd.it</a><br/>
University of Padua, Italy<br/>
2021/2022<br/>

# Lecture 12: The Regression Problem - Example (Part 2)

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import stats module from scipy, which contains a large number 
# of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats
# Import the sub-module 'linear_model' of scikit-learn
# this sub-module implements generalized linear models.
# API: http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
from sklearn import linear_model
# Import two quality metrics which will be used to assess the quality
# of our predictions: MSE and R squared (a.k.a. coefficient of determination)
from sklearn.metrics import mean_squared_error, r2_score

%matplotlib inline

# Summary of Part 1

In [ ]:
# Path to the local dataset file
DATASET_PATH = "./data/auto-mpg-regression/dataset.tsv"

## Loading Dataset

In [ ]:
columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 
           'acceleration', 'model_year', 'origin', 'car_name']

data = pd.read_csv(DATASET_PATH, sep="\t", header=None, 
                   names=columns,
                  na_values={'horsepower':'?'})

print("Shape of the dataset: {}".format(data.shape))
data.head()

## Change the Column Layout

In [ ]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data.columns.tolist()
# Popping out 'mpg' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('mpg')))
# Let's refactor the DataFrame using this new column index
data = data.loc[:, columns]
data.head()
# Alternatively to 'loc' we can also use 'reindex()'
# data = data.reindex(columns=columns)
# data.head()

## Apply Some Simple Data Transformations

In [ ]:
# Suppose we want to convert displacement unit from cubic inch to litre
# There is a useful conversion table which tells us how to do that.
# 1 cubic inch = 0.016387064 litre
CI_TO_LITRE = 0.016387064
data.displacement *= CI_TO_LITRE
data.head()

## Handling Missing Values (NA)

In [ ]:
# Let's replace missing values on 'horsepower'
# using the median as computed from the other observations.
# NOTE: here's a classical example where using the mean rather than the median
# might affect the result, as the mean is more sensitive to outliers.
# NOTE: by default, median() does not include NAs in the computation.
# In other words, we don't need to explicitly tell pandas to work on non-NA values:
# data.horsepower[data.horsepower.notnull()].median()
data.horsepower.fillna(data.horsepower.median(), inplace=True)
data.head()

## Handling Outliers

In [ ]:
# Let's winsorize 'horsepower' and 'acceleration'
_ = stats.mstats.winsorize(data.horsepower, limits=0.0375, inplace=True)
_ = stats.mstats.winsorize(data.acceleration, limits=0.0375, inplace=True)

## Encoding Categorical Features: One-Hot Encoding

In [ ]:
# In pandas we can achieve easily one-hot encoding using the 'get_dummies()' function
categorical_features = ['cylinders', 'model_year', 'origin']
data = pd.get_dummies(data, columns = categorical_features)
data.head()

In [ ]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data.columns.tolist()
# Popping out 'mpg' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('mpg')))
# Let's refactor the DataFrame using this new column index
data = data.loc[:, columns]
data.head()

In [ ]:
def sanitize_automaker_name(car_name):
    s = car_name.split(' ')[0]
    if s == 'vw' or s == 'vokswagen':
        return car_name.replace(s,'volkswagen')
    if s == 'chevroelt' or s == 'chevy':
        return car_name.replace(s,'chevrolet')
    if s == 'maxda':
        return car_name.replace(s,'mazda')
    if s == 'mercedes':
        return car_name.replace(s,'mercedes-benz')
    if s == 'toyouta':
        return car_name.replace(s,'toyota')
    return car_name

In [ ]:
"""
Let's sanitize automaker name on 'car_name' column
"""
data['car_name'] = data['car_name'].map(lambda x: sanitize_automaker_name(x))
data.head()

In [ ]:
"""
Three solutions can be designed to tackle with this issue:
1) Just drop the column 'car_name' (i.e., our model won't rely on that feature for prediction)
2) Use one-hot encoding scheme and deal with sparsity data (i.e., possibly leading to overfitting)
3) Trade-off: try to build another column which somehow reduces (i.e., cluster) similar values together
and then apply one-hot encoding.
Let's see how to perform 3)
"""
# Suppose we want to create another column called 'automaker_name', which simply contains
# the name of the automaker, disregarding the model.
# For example, automaker_name('ford gran torino') = automaker_name('ford f250') = 'ford'
data['automaker_name'] = data['car_name'].map(lambda x: x.split(' ')[0])
data.head()

In [ ]:
# Create the set of the top-10 automakers
top_10_automakers = set(data['automaker_name'].value_counts()[:10].index)
# Label with 'other' any automaker_name which is not in the list above
data['automaker_name'] = np.where(data['automaker_name'].isin(top_10_automakers), 
                                               data['automaker_name'],
                                              'other')

In [ ]:
categorical_features = ['automaker_name']
data = pd.get_dummies(data, columns = categorical_features)
data.head()

In [ ]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data.columns.tolist()
# Popping out 'mpg' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('mpg')))
# Popping out 'automaker_name_other' from the list and insert it after 'automaker_name_volkswagen'.
columns.insert(columns.index('automaker_name_volkswagen'), columns.pop(columns.index('automaker_name_other')))
# Popping out 'car_name' from the list and insert it right before 'mpg'
columns.insert(-1, columns.pop(columns.index('car_name')))
# Let's refactor the DataFrame using this new column index
data = data.loc[:, columns]
data.head()

# 4. Building a Predictive Model

In [ ]:
data.head()

# 4.1 Dataset Splitting: _Training_ vs. _Test_

In [ ]:
"""
Extract the feature matrix from our original DataFrame.
"""
# Feature matrix X is composed of all the columns 
# except 'car_name' and 'mpg' (i.e., the target)
X = data.iloc[:,:-2]
X.head()

In [ ]:
"""
Similarly, we want to extract the target column vector y.
"""
y = data.mpg
y.head()

Let's split our dataset with scikit-learn 'train_test_split' function, 
which splits the input dataset into training and test set, respectively.
We want the training set to account for 80% of the original dataset, whilst 
the test set to account for the remaining 20%.
Additionally, we would like to take advantage of stratified sampling,
so as to obtain the same target distribution in both the training and the test sets.
PROBLEM: When target (y) is categorical this is straightforward (just use 'stratify=y').
If y is continous, instead, we need to 'discretize' it before use stratified sampling.

In [ ]:
# 1. Check the number of observations contained in 'y'.
print("N. of observations = {}".format(y.shape[0]))

# 2. Suppose we want to use the Friedman-Diaconis rule to devise the number of bins.
# bin_width = 2 * IQR/n^{1/3}; n_bins = (max - min)/bin_width
y_q1, y_q3 = y.quantile([.25, .75])
print("1st quartile = {:.2f}".format(y_q1))
print("3rd quartile = {:.2f}".format(y_q3))
y_IQR = (y_q3 - y_q1)
bin_width = 2 * y_IQR/math.pow(y.shape[0], 1/3)
n_bins = int((y.max() - y.min())//bin_width)
print("N. of bins = {}".format(n_bins))

# 3. Save our y values in a new ndarray,
# broken down by the bins created above.

#bins = np.linspace(0, y.shape[0], n_bins)

bins = np.linspace(y.min(), y.max(), n_bins)

y_binned = np.digitize(y, bins)

In [ ]:
unique, counts = np.unique(y_binned, return_counts=True)
counts

In [ ]:
from sklearn.model_selection import train_test_split
# 4. Pass y_binned to the stratify argument,
# and sklearn will handle the rest
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y_binned)

In [ ]:
bins[:-1]

In [ ]:
bins = np.linspace(y.min(), y.max(), n_bins)

y_binned = np.digitize(y, bins[:-1])

In [ ]:
unique, counts = np.unique(y_binned, return_counts=True)
counts

In [ ]:
from sklearn.model_selection import train_test_split
# 4. Pass y_binned to the stratify argument,
# and sklearn will handle the rest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y_binned)

In [ ]:
print("Training Set shape: {}".format(X_train.shape))
print("Test Set shape: {}".format(X_test.shape))

## Feature Scaling: Why/When

-  <span style="color: red">**REMEMBER:**</span> Not every learning models are sensitive to different feature scales! 

-  For example, in the case of Linear Regression the vector of model parameters we come up with when we minimize the MSE - using either the pseudo-inverse (closed-form) or gradient descent (iterative) solution - is **not** affected by different feature scales, except for a constant.

-  You can convince yourself of this by computing the gradient of MSE using non-scaled and scaled features.

-  Other models, instead, are not invariant with respect to scalar transformations of the input (features), and leads to completely different results if features are not properly scaled. 

## Feature Scaling: How

-  Feature scaling **cannot** be done looking at the whole dataset!

-  In other words, either you standardize (using **z-scores**) or normalize (using **min-max**) your features you **must** do it considering only the training set portion of your dataset.

-  The same scaling, then, should be applied to the test set.

In [ ]:
"""
Let's use two different feature scaling strategies: standard z-scores and min-max
"""
# The following is the scikit-learn package which provides
# various preprocessing capabilities
from sklearn import preprocessing

In [ ]:
# Standardizing features using z-score
std_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_mean = X_train.mean()
# X_train_std = X_train.std()
# X_train_std = (X_train - X_train_mean)/X_train_std

# Normalizing features using min-max
minmax_scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_minmax = minmax_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_max = X_train.max()
# X_train_min = X_train.min()
# X_train_minmax = (X_train - X_train_min)/(X_train_max - X_train_min)

In [ ]:
"""
At this stage, we can work with 3 different feature matrices:
- The original one: X_train
- The standardized one: X_train_std
- The min-max normalized one: X_train_minmax
"""

In [ ]:
"""
General function used to fit a given model to a training set (X_train, y_train)
"""
def fit(model, X_train, y_train):
    model.fit(X_train, y_train)

In [ ]:
"""
General function used to make predictions on the test set (X_test)
"""
def predictions(model, X_test):
    return model.predict(X_test)

In [ ]:
"""
General function used to assess the quality of predictions
in terms of two scores: MSE and R2
"""
def evaluate(true_values, predicted_values):
    # The mean squared error
    print("Mean Squared Error (MSE) = {:.6f}".
          format(mean_squared_error(true_values, predicted_values)))
    
    # Explained variance score: 1 is perfect prediction
    print("Coefficient of Determination (R2 score) = {:.6f}".
          format(r2_score(true_values, predicted_values)))

In [ ]:
def print_regression_coefficients(model, columns):
    
    print("Regression Coefficients:\n{}".format("\n".join(["["
                                                           + columns[i].strip() 
                                                           + "={:.2f}".
                                                           format(model.coef_[i]) 
                                                           +"]" 
                                                           for i in range(0, len(columns))]
                                                         )
                                               )
         )

In [ ]:
def plot_true_vs_predicted(y_true, y_predicted):
    fig, ax = plt.subplots()
    ax.scatter(y_true, y_predicted, edgecolors=(0, 0, 0), color='red')
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=4)
    ax.set_xlabel('True MPG')
    ax.set_ylabel('Predicted MPG')

In [ ]:
# Create linear regression object
model = linear_model.LinearRegression()

# 1. Try to fit this linear regressor to our original training set
fit(model, X_train, y_train)

# 2. Assess the quality of predictions made on the same training set
print("***** Evaluate Predictions on Training Set *****")

mpg_y_train_pred = model.predict(X_train)
evaluate(y_train, mpg_y_train_pred)
plot_true_vs_predicted(y_train, mpg_y_train_pred)
print()

# 3. Assess the quality of predictions made on the test set
print("***** Evaluate Predictions on Test Set *****")
mpg_y_test_pred = model.predict(X_test)
evaluate(y_test, mpg_y_test_pred)
plot_true_vs_predicted(y_test, mpg_y_test_pred)

In [ ]:
# Create another linear regression object
model_std = linear_model.LinearRegression()

# 1. Try to fit this linear regressor to our original training set
fit(model_std, X_train_std, y_train)
# 2. Scale the test set (using the same scaler fit to the training set)
X_test_std = std_scaler.transform(X_test)
#X_test_std = (X_test - X_train_mean)/X_train_std
# 3. Make predictions using the testing set
mpg_y_pred_std = model_std.predict(X_test_std)
# 4. Evaluate predictions
evaluate(y_test, mpg_y_pred_std)
# 5. Plot true vs. predicted values of 'mpg'
plot_true_vs_predicted(y_test, mpg_y_pred_std)

In [ ]:
# Create another linear regression object
model_minmax = linear_model.LinearRegression()

# 1. Try to fit this linear regressor to our original training set
fit(model_minmax, X_train_minmax, y_train)
# 2. Scale the test set (using the same scaler fit to the training set)
X_test_minmax = minmax_scaler.transform(X_test)
#X_test_minmax = (X_test - X_train_min)/X_train_max
# 3. Make predictions using the testing set
mpg_y_pred_minmax = model_minmax.predict(X_test_minmax)
# 4. Evaluate predictions
evaluate(y_test, mpg_y_pred_minmax)
# 5. Plot true vs. predicted values of 'mpg'
plot_true_vs_predicted(y_test, mpg_y_pred_minmax)

## $k$-fold Cross Validation

In [ ]:
"""
Instead of splitting training and test set, we can use k-fold cross validation.
NOTE: Since there is no need for hyperparameter tuning here,
we can simply run k-fold cross validation on the whole dataset (X)
'cross_val_score' is a function which gives us the cv scores
(R2 is the default score for linear regression).
"""
from sklearn.model_selection import cross_val_score

# Reset the linear regression object
cv_model = linear_model.LinearRegression()

# Run 'cross_val_score' to get the R2 score on each fold
# NOTE: cross validation is performed on the whole dataset as there is no
# need for hyperparameter tuning at this stage, nor model comparison.
cv_scores = cross_val_score(cv_model, X, y, cv=10) 
                            #scoring="neg_mean_squared_error")

print("***** Cross Validation Summary *****")
for v in range(0, len(cv_scores)):
    print("R2 score on fold-{} = [{}]".format(v+1, cv_scores[v]))

In [ ]:
import sklearn

sklearn.metrics.get_scorer_names()

In [ ]:
print("Average 10-fold cross validation R2 score: {:.2f} (+/- {:.2f})"
      .format(cv_scores.mean(), cv_scores.std()))

In [ ]:
"""
By default, no shuffling is performed before data splitting.
We can override this behavior using a specific instance of KFold class.
"""
from sklearn.model_selection import KFold
# Reset the linear regression object
cv_model = linear_model.LinearRegression()

# Run 'cross_val_score' to get the R2 score on each fold
cv_scores = cross_val_score(cv_model, X, y, cv=KFold(n_splits=10, 
                                                     shuffle=True, random_state=42))

print("***** Cross Validation Summary *****")
for v in range(0, len(cv_scores)):
    print("R2 score on fold-{} = [{}]".format(v+1, cv_scores[v]))

In [ ]:
print("Average 10-fold cross validation R2 score (with random shuffling): {:.2f} (+/- {:.2f})"
      .format(cv_scores.mean(), cv_scores.std()))

In [ ]:
"""
Instead of splitting training and test set, we can use k-fold cross validation.
NOTE: Since there is no need for hyperparameter tuning here,
we can simply run k-fold cross validation on the whole dataset (X)
'cross_validate' is a function which gives us a dictionary of scores.
"""
from sklearn.model_selection import cross_validate
# Reset the linear regression object
cv_model = linear_model.LinearRegression()

scoring = ['r2']
cv_scores = cross_validate(cv_model, X, y, scoring=scoring, 
                           cv=KFold(n_splits=10, shuffle=True, random_state=42), 
                           return_train_score=True)
print("***** Cross Validation Summary *****")
for k in cv_scores:
    print("{} = [{}]".format(k, ", ".join(["{:.5f}".format(v) for v in cv_scores[k]])))

In [ ]:
"""
Use the outcome of k-fold cross validation to generate predictions, 
calling 'cross_val_predict' function.
"""
from sklearn.model_selection import cross_val_predict

# Reset the linear regression object
cv_model = linear_model.LinearRegression()
# We generate predictions on the basis of the result of 10-fold cv
mpg_y_pred_cv = cross_val_predict(cv_model, X, y, cv=KFold(n_splits=10, 
                                                           shuffle=True, random_state=42))
# Evaluate predictions
evaluate(y, mpg_y_pred_cv)
# Plot true vs. predicted values (using 10-fold cross validation)
plot_true_vs_predicted(y, mpg_y_pred_cv)

In [ ]:
"""
Leave-One-Out cross validation (i.e., k = n, number of observations)
"""
from sklearn.model_selection import cross_val_predict

# Reset the linear regression object
cv_model = linear_model.LinearRegression()
# We generate predictions on the basis of the result of leave-one-out
mpg_y_pred_cv = cross_val_predict(cv_model, X, y, cv=X.shape[0])
# Evaluate predictions
evaluate(y, mpg_y_pred_cv)
# Plot true vs. predicted values (using leave-one-out cross validation)
plot_true_vs_predicted(y, mpg_y_pred_cv)